In [1]:
import pandas as pd
from copy import deepcopy
import os
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from tqdm import tqdm
from glob import glob

import pwlf

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def canbefloat(x):
    try:
        float(x)
    except:
        return False
    else:
        return True
ucanbefloat = lambda t: canbefloat(t)
vfunc = np.vectorize(ucanbefloat)

In [5]:
file = '/scratch/ab9738/traffic/data/Nairobi/per-segment/555497908.csv'

In [7]:
cols = pd.read_csv('/scratch/ab9738/traffic/data/Sao_Paulo/movement-speeds-hourly-sao-paulo-2018-1.csv',nrows=1).columns
df = pd.read_csv(file,on_bad_lines='skip',header=None,low_memory=False)
df.columns = cols

In [9]:
df = df.dropna()

timestamp = df['utc_timestamp']
timestamp = pd.to_datetime(timestamp, format="%Y-%m-%dT%H:%M:%S.000Z", errors='coerce')
df = df.assign(utc_timestamp=timestamp)
df = df.dropna()
df = df.sort_values(by='utc_timestamp')

In [15]:
df.iloc[:20]

,year,month,day,hour,utc_timestamp,segment_id,start_junction_id,end_junction_id,osm_way_id,osm_start_node_id,osm_end_node_id,speed_kph_mean,speed_kph_stddev
638,2018,1,9,17,2018-01-09 14:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,29.632,27.598
631,2018,1,10,8,2018-01-10 05:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,29.965,9.986
642,2018,1,10,16,2018-01-10 13:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,14.997,5.889
641,2018,1,10,17,2018-01-10 14:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,28.289,4.664
633,2018,1,11,9,2018-01-11 06:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,28.071,12.157
644,2018,1,11,16,2018-01-11 13:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,21.522,5.500
636,2018,1,12,16,2018-01-12 13:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,14.747,7.527
645,2018,1,15,9,2018-01-15 06:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,22.519,5.517
643,2018,1,15,16,2018-01-15 13:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,12.179,3.850
632,2018,1,15,17,2018-01-15 14:00:00,79196135947dd799dd4be01ff5eb9a0cbc7111a5,a43315d490dee6b074756710e71558fc1586b907,92c8ce431867324ee7ada8b62bd871a91f9bdb3c,555497908,2985990955,5359406855,11.461,3.619


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import pwlf
from scipy.signal import savgol_filter

## Load data and process time

In [19]:
df = pd.read_csv('/scratch/ab9738/traffic/data/NYCDOT/toi_file.csv')

In [20]:
timestamp = df['DATA_AS_OF']
timestamp = pd.to_datetime(timestamp, format="%m/%d/%Y %I:%M:%S %p")
df['DATA_AS_OF'] = timestamp

Remove cases when speed is 0 since it means data is not available.

In [21]:
df = df[df.SPEED != 0]

In [22]:
road_ids = df['ID'].unique()

In [23]:
len(road_ids)

122

In [24]:
df = df.set_index('ID')

Sort by date

In [25]:
df = df.sort_values('DATA_AS_OF')

In [27]:
df = df[['SPEED','DATA_AS_OF','TRAVEL_TIME']]

In [36]:
for i in range(len(road_ids)):
    arr_s = df.loc[road_ids[i]]['SPEED'].to_numpy() * df.loc[road_ids[i]]['TRAVEL_TIME'].to_numpy()
    print(np.std(arr_s)/np.mean(arr_s))

0.014835716137876103
0.036761518356301676
0.0516803847144051
0.02943129207148471
0.11686231064944398
0.03728702273398424
0.011333639497839724
0.01885448446103383
0.00999039311906536
0.022398413652746602
0.010598784510529598
0.008929627216819058
0.007315822490206015
0.02961604668846363
0.007672738657295448
0.029539397221549298
0.008046113311077619
0.006570618155118735
0.025167485659874374
0.02369867660709657
0.01332299367515644
0.011470991642228513
0.09141182537144023
0.07331998927387051
0.00915323894821663
0.013073086082462498
0.018167107624268243
0.013479566435174313
0.048061201187462134
0.015829458025298126
0.012258688828835126
0.029670495408177438
0.02072781569936983
0.021829338262726893
0.015225441377078004
0.01048003303470247
0.008016441898351306
0.005167820906226162
0.00485248647210191
0.5069623119075024
0.016551774863432494
0.017140573933349064
0.024857925324220712
0.006313895406878994
0.006310208511712021
0.018340020975596064
0.008325359156483544
0.011407854505045228
0.00868082

In [74]:
df = pd.read_csv('/scratch/ab9738/traffic/data/sao_paulo-od_zones_2017-2020-1-All-DatesByHourBucketsAggregate.csv')

In [75]:
df = df.set_index(['sourceid', 'dstid'])

In [77]:
srcids = df.index.get_level_values('sourceid').unique().to_numpy()

In [78]:
tt_blow = []
for src in srcids:
    df_ = df.loc[src]
    dstids = df_.index.unique().to_numpy()
    for dst in dstids:
        df_ = df.loc[src,dst]
        tt = df_['mean_travel_time'].to_numpy()
        # print(np.mean(tt), np.max(tt), np.max(tt)/np.mean(tt))
        tt_blow.append(np.max(tt)/np.mean(tt))
tt_blow = np.array(tt_blow)

KeyboardInterrupt: 

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.figure(figsize=(8,5))
plt.hist(tt_blow, bins=np.arange(24)*0.5, edgecolor='black', weights=np.ones_like(tt_blow)/len(tt_blow))
plt.title('Travel Time Blow-Up for Segments: Sao Paulo')
plt.xlabel('Max increase in travel time')
plt.ylabel('Relative Frequency')